In [33]:
import os
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [51]:
IMAGE_TRAIN_DIR = '../data/image/train/'
IMAGE_VALIDATION_DIR = '../data/image/validation/'
VIDEO_TRAIN_DIR = '../data/videos/train/'
VIDEO_VAL_DIR = '../data/videos/validation/'

## Preprocessing untuk Gambar

In [52]:
# Data Augmentation untuk gambar pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisasi gambar
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [36]:
# Hanya normalisasi untuk data validasi
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    IMAGE_TRAIN_DIR,
    target_size=(224, 224),
    batch_size=32,  # Anda bisa mencoba ukuran batch lebih kecil jika diperlukan
    class_mode='sparse'
)

validation_generator = valid_datagen.flow_from_directory(
    IMAGE_VALIDATION_DIR,
    target_size=(224, 224),
    batch_size=32,
    class_mode='sparse'
)


Found 2851 images belonging to 26 classes.
Found 2851 images belonging to 26 classes.


## Preprocessing untuk Video

In [82]:
# Fungsi untuk mengambil frame dari video
def extract_frames_from_video(video_path, num_frames=30, target_size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = frame_count // num_frames

    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, target_size)
            frame = frame / 255.0  # Normalisasi
            frames.append(np.float32)

    cap.release()
    return np.array(frames)


# Fungsi untuk mendapatkan semua video dan label berdasarkan subfolder
def get_video_files_and_labels(video_dir):
    video_files = []
    labels = {}
    for subfolder in os.listdir(video_dir):
        subfolder_path = os.path.join(video_dir, subfolder)
        if os.path.isdir(subfolder_path):
            # Dapatkan semua file video di dalam subfolder
            for file in os.listdir(subfolder_path):
                if file.endswith(".mp4"):  # Pastikan hanya file .mp4 yang diproses
                    video_path = os.path.join(subfolder_path, file)
                    video_files.append(video_path)
                    labels[video_path] = subfolder  # Gunakan nama subfolder sebagai label
    return video_files, labels

# Fungsi generator untuk video dengan label
import itertools

# Fungsi generator untuk video dengan label
def video_data_generator(video_dir, labels, batch_size=16, num_frames=30, target_size=(224, 224)):
    video_files, _ = get_video_files_and_labels(video_dir)
    while True:
        batch_videos = []
        batch_labels = []
        for video_path in itertools.islice(video_files, 0, batch_size):
            frames = extract_frames_from_video(video_path, num_frames, target_size)
            label = labels.get(video_path)  # Ambil label berdasarkan path video
            batch_videos.append(frames)
            batch_labels.append(label)

        batch_videos = np.array(batch_videos)
        batch_labels = np.array(batch_labels)
        yield batch_videos, batch_labels  # Mengembalikan tuple (data, label)


# Membaca video dari direktori pelatihan dan validasi
VIDEO_TRAIN_DIR = '../data/videos/train/'
VIDEO_VAL_DIR = '../data/videos/validation/'  # Pastikan Anda memiliki direktori validasi

# Dapatkan semua file video dan label berdasarkan subfolder
video_train_files, labels_train = get_video_files_and_labels(VIDEO_TRAIN_DIR)
video_val_files, labels_val = get_video_files_and_labels(VIDEO_VAL_DIR)

# Menghitung langkah per epoch dan langkah validasi
steps_per_epoch = len(video_train_files) // 32  # batch_size = 32
validation_steps = len(video_val_files) // 32  # batch_size = 32

# Buat generator untuk pelatihan dan validasi
train_video_generator = video_data_generator(VIDEO_TRAIN_DIR, labels_train, batch_size=16)
validation_video_generator = video_data_generator(VIDEO_VAL_DIR, labels_val, batch_size=16)


print(f"Train generator and validation generator prepared.")

Train generator and validation generator prepared.


## Membangun Model CNN

In [37]:
# Membangun model CNN untuk pengenalan gambar
model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),  # Menyatakan bentuk input secara eksplisit
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Jumlah kelas
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan data pelatihan
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

Epoch 1/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 148s 2s/step - accuracy: 0.1693 - loss: 3.0095 - val_accuracy: 0.7611 - val_loss: 0.9157
Epoch 2/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.8067 - loss: 0.7507 - val_accuracy: 0.9197 - val_loss: 0.3342
Epoch 3/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 149s 2s/step - accuracy: 0.9403 - loss: 0.2306 - val_accuracy: 0.9835 - val_loss: 0.0928
Epoch 4/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9782 - loss: 0.0765 - val_accuracy: 0.9909 - val_loss: 0.0583
Epoch 5/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 137s 2s/step - accuracy: 0.9903 - loss: 0.0412 - val_accuracy: 0.9982 - val_loss: 0.0131
Epoch 6/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 134s 1s/step - accuracy: 0.9944 - loss: 0.0205 - val_accuracy: 0.9930 - val_loss: 0.0261
Epoch 7/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.9935 - loss: 0.0346 - val_accuracy: 0.9989 - val_loss: 0.0055
Epoch 8/10
90/90 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.9983 - loss: 0.0086 - val_accuracy: 0.9996 - v

In [38]:
# Evaluasi model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

90/90 ━━━━━━━━━━━━━━━━━━━━ 57s 634ms/step - accuracy: 0.9998 - loss: 0.0013
Validation Accuracy: 99.96%


## Training Video

In [76]:
print(f"Jumlah video pelatihan: {len(video_train_files)}")
print(f"Jumlah video validasi: {len(video_val_files)}")


Jumlah video pelatihan: 52
Jumlah video validasi: 52


In [83]:
# Membangun model CNN 3D untuk pengenalan bahasa isyarat dalam video
model = models.Sequential([
    layers.Input(shape=(30, 224, 224, 3)),  # 30 frame, resolusi 224x224, 3 saluran warna
    layers.Conv3D(32, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Conv3D(64, (3, 3, 3), activation='relu'),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(labels_train), activation='softmax')  # Jumlah kelas sesuai jumlah label
])

# Menyusun model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melatih model dengan data video
history = model.fit(
    train_video_generator,
    validation_data=validation_video_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

ValueError: Invalid dtype: object